In [ ]:
# %load train.py
#!/usr/bin/env python

# In[1]:
from t_cnn import tcnn

model = tcnn(num_classes = 2,pretrained = True, model_root = '/home/krf/model/BALL/')

import senet
import os
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from utils import TransformImage
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tensorboardX import SummaryWriter

DATA_DIR = "/home/krf/dataset/BALL/"
traindir = DATA_DIR + "train"
valdir = DATA_DIR +"val"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
BATCH_SIZE = 32
WORKERS = 4
START = 0
EPOCHS = 400
PRINT_FREQ = 10


# In[31]:


#model =  senet.se_resnext50_32x4d(num_classes = 2)
#通过随机变化来进行数据增强
train_tf  = TransformImage(
    model,
    
    random_crop=False,
    random_hflip=True,
    random_vflip=True,
    random_rotate=True,
    preserve_aspect_ratio=True
)
train_loader = torch.utils.data.DataLoader(
#     datasets.ImageFolder(traindir, transforms.Compose([
# #         transforms.RandomSizedCrop(max(model.input_size)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         normalize,
#     ])),
    datasets.ImageFolder(traindir,train_tf),
    batch_size=BATCH_SIZE, shuffle=True,
    num_workers=WORKERS, pin_memory=True)


val_tf = TransformImage(
    model,
    
    preserve_aspect_ratio=True)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir,val_tf),
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=WORKERS, pin_memory=True)


# In[29]:


def train(train_loader, model, criterion, optimizer, epoch,scheduler):
    # switch to train mode
    model.train()
#     end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
#         data_time.update(time.time() - end)
        input = input.cuda()
        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input.float())
        target_var = torch.autograd.Variable(target)
        #print(input_var.type())
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        #print(output.data)



#         # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()
        meters = trainMeter.update(output,target,loss,input.size(0))

        if i % PRINT_FREQ == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss:.5f}\t'
                  'Acc {Acc:.5f}\t'
                  'Precision {P:.5f}\t'
                  'Recall {R:.5f}\t'
                  'F1 {F1:.5f}'.format(
                   epoch,i, len(train_loader), loss=meters[4],
                   Acc=meters[3],P=meters[0],R=meters[1],F1=meters[2]))
            
            step = epoch*len(train_loader) + i
            
            writer.add_scalar('TRAIN/Precision', meters[0], step)
            writer.add_scalar('TRAIN/Recall', meters[1], step)
            writer.add_scalar('TRAIN/F1', meters[2], step)
            writer.add_scalar('TRAIN/Acc', meters[3], step)
            writer.add_scalar('TRAIN/loss',meters[4], step)
            
    scheduler.step(meters[4])


def validate(val_loader, model, criterion,epoch):
    # switch to evaluate mode
    model.eval()
    
#     end = time.time()
    meters = []
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        meters = valMeter.update(output,target,loss,input.size(0))
        if i % PRINT_FREQ == 0:
            print('Test: [{0}/{1}]\t'
                  'Loss {loss:.5f}\t'
                  'Acc {Acc:.5f}\t'
                  'Precision {P:.5f}\t'
                  'Recall {R:.5f}\t'
                  'F1 {F1:.5f}'.format(
                   i, len(val_loader), loss=meters[4],
                   Acc=meters[3],P=meters[0],R=meters[1],F1=meters[2]))
            
            step = epoch * len(val_loader) + i
            writer.add_scalar('VAL/Precision', meters[0], step)
            writer.add_scalar('VAL/Recall', meters[1], step)
            writer.add_scalar('VAL/F1', meters[2], step)
            writer.add_scalar('VAL/Acc', meters[3], step)
            writer.add_scalar('VAL/loss',meters[4], step)
    print(' * Acc {Acc:.5f} F1 {F1:.5f}'
          .format(Acc=meters[3],F1=meters[2]))
    writer.add_scalar('VAL/EPOCH_F1', meters[2], epoch)
    return meters[2]


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

class ModelMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.losses = AverageMeter()
        self.top1 = AverageMeter()
        self.TP = 0
        self.TN = 0
        self.FN = 0
        self.FP = 0
        self.P=0
        self.R=0
        self.F1=0
        self.Acc=0

    def update(self, output,target,loss, n=1):
        _, pred = output.data.topk(1, 1, True, True)
        pred = pred.t()
#         print(pred,target.data)
        # TP    predict 和 label 同时为1
        self.TP += ((pred == 1) & (target.data == 1)).cpu().numpy().sum()
        # TN    predict 和 label 同时为0
        self.TN += ((pred == 0) & (target.data == 0)).cpu().numpy().sum()
        # FN    predict 0 label 1
        self.FN += ((pred == 0) & (target.data == 1)).cpu().numpy().sum()
        # FP    predict 1 label 0
        self.FP += ((pred == 1) & (target.data == 0)).cpu().numpy().sum()
#         print(self.TP,self.TN,self.FN,self.FP)
        self.P = self.TP / (self.TP + self.FP)
        self.R = self.TP / (self.TP + self.FN)
        self.F1 = 2 * self.R * self.P / (self.R + self.P)
        
        self.Acc = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        
        self.losses.update(loss.data[0],n)

        return [self.P,self.R,self.F1,self.Acc,self.losses.avg]

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


# def adjust_learning_rate(optimizer, epoch):
#     """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
#     lr = args.lr * (0.1 ** (epoch // 30))
#     for param_group in optimizer.param_groups:
#         param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


# In[ ]:

# 加载模型，解决命名和维度不匹配问题,解决多个gpu并行
def load_state_keywise(model, model_path):
    model_dict = model.state_dict()
    
    print("=> loading checkpoint '{}'".format(model_path))
    checkpoint = torch.load(model_path,map_location='cpu')
    START = checkpoint['epoch']
    best_F1 = checkpoint['best_prec1']
    #model.load_state_dict(checkpoint['state_dict'])
    
    pretrained_dict = checkpoint['state_dict']#torch.load(model_path, map_location='cpu')
    key = list(pretrained_dict.keys())[0]
    # 1. filter out unnecessary keys
    # 1.1 multi-GPU ->CPU
    if (str(key).startswith('module.')):
        pretrained_dict = {k[7:]: v for k, v in pretrained_dict.items() if
                           k[7:] in model_dict and v.size() == model_dict[k[7:]].size()}
    else:
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if
                           k in model_dict and v.size() == model_dict[k].size()}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return START,best_F1



criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2,momentum = 0.9,weight_decay=1e-6)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)

# START,best_f1 = load_state_keywise(model,'checkpoint.pth.tar')
# model = model.cuda()
model = torch.nn.DataParallel(model).cuda()

# TP = 0,TN = 0,FN = 0, FP = 0
writer = SummaryWriter()
best_f1 = 0
trainMeter = ModelMeter()
valMeter = ModelMeter()
for epoch in range(START,EPOCHS):
    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, scheduler)
    # evaluate on validation set
    F1 = validate(val_loader, model, criterion,epoch)
    
    # remember best prec@1 and save checkpoint
    is_best = F1 > best_f1
    best_f1 = max(F1, best_f1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "T-CNN",
        'state_dict': model.state_dict(),
        'best_prec1': best_f1,
    }, is_best)
# export scalar data to JSON for external processing
writer.export_scalars_to_json("./test.json")
writer.close()







/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:200: RuntimeWarning: invalid value encountered in double_scalars
/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:201: RuntimeWarning: invalid value encountered in double_scalars
/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:204: RuntimeWarning: invalid value encountered in double_scalars
/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:206: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: [0][0/278]	Loss 6.91405	Acc nan	Precision nan	Recall nan	F1 nan
Epoch: [0][10/278]	Loss 6.81572	Acc 0.71168	Precision nan	Recall 0.00000	F1 nan
Epoch: [0][20/278]	Loss 6.58653	Acc 0.72391	Precision nan	Recall 0.00000	F1 nan
Epoch: [0][30/278]	Loss 5.76707	Acc 0.68709	Precision 0.32812	Recall 0.07955	F1 0.12805
Epoch: [0][40/278]	Loss 5.44820	Acc 0.67909	Precision 0.32292	Recall 0.08564	F1 0.13537
Epoch: [0][50/278]	Loss 5.23110	Acc 0.65701	Precision 0.30208	Recall 0.12691	F1 0.17874
Epoch: [0][60/278]	Loss 5.03595	Acc 0.66275	Precision 0.29911	Recall 0.12362	F1 0.17493
Epoch: [0][70/278]	Loss 4.50258	Acc 0.66272	Precision 0.30859	Recall 0.12305	F1 0.17595
Epoch: [0][80/278]	Loss 4.09323	Acc 0.66667	Precision 0.31250	Recall 0.12329	F1 0.17682
Epoch: [0][90/278]	Loss 3.72702	Acc 0.66126	Precision 0.29664	Recall 0.11729	F1 0.16811
Epoch: [0][100/278]	Loss 3.43316	Acc 0.65346	Precision 0.27473	Recall 0.10764	F1 0.15468
Epoch: [0][110/278]	Loss 3.18632	Acc 0.65141	Precision 0.27179	R

/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:138: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:139: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test: [0/21]	Loss 0.40487	Acc 1.00000	Precision nan	Recall nan	F1 nan
Test: [10/21]	Loss 0.42562	Acc 1.00000	Precision nan	Recall nan	F1 nan
Test: [20/21]	Loss 0.65343	Acc 0.60301	Precision nan	Recall 0.00000	F1 nan
 * Acc 0.60301 F1 nan
Epoch: [1][0/278]	Loss 1.63756	Acc 0.68054	Precision 0.26025	Recall 0.04903	F1 0.08252
Epoch: [1][10/278]	Loss 1.60109	Acc 0.68166	Precision 0.26025	Recall 0.04735	F1 0.08013
Epoch: [1][20/278]	Loss 1.56796	Acc 0.68386	Precision 0.26025	Recall 0.04596	F1 0.07813
Epoch: [1][30/278]	Loss 1.53751	Acc 0.68439	Precision 0.26025	Recall 0.04442	F1 0.07589
Epoch: [1][40/278]	Loss 1.50819	Acc 0.68547	Precision 0.26025	Recall 0.04307	F1 0.07390
Epoch: [1][50/278]	Loss 1.47997	Acc 0.68582	Precision 0.26025	Recall 0.04169	F1 0.07187
Epoch: [1][60/278]	Loss 1.45537	Acc 0.68717	Precision 0.26653	Recall 0.04245	F1 0.07324
Epoch: [1][70/278]	Loss 1.43166	Acc 0.68538	Precision 0.28253	Recall 0.05105	F1 0.08648
Epoch: [1][80/278]	Loss 1.40853	Acc 0.68535	Precision 0.283

Test: [0/21]	Loss 0.64396	Acc 0.63049	Precision 1.00000	Recall 0.05429	F1 0.10299
Test: [10/21]	Loss 0.58772	Acc 0.67916	Precision 0.91489	Recall 0.05429	F1 0.10250
Test: [20/21]	Loss 0.64576	Acc 0.65075	Precision 0.87135	Recall 0.14110	F1 0.24287
 * Acc 0.65075 F1 0.24287
Epoch: [4][0/278]	Loss 0.81834	Acc 0.72122	Precision 0.56269	Recall 0.21915	F1 0.31544
Epoch: [4][10/278]	Loss 0.81550	Acc 0.72162	Precision 0.56334	Recall 0.22002	F1 0.31645
Epoch: [4][20/278]	Loss 0.81220	Acc 0.72223	Precision 0.56504	Recall 0.22097	F1 0.31770
Epoch: [4][30/278]	Loss 0.80910	Acc 0.72294	Precision 0.56860	Recall 0.22356	F1 0.32094
Epoch: [4][40/278]	Loss 0.80600	Acc 0.72358	Precision 0.57150	Recall 0.22554	F1 0.32344
Epoch: [4][50/278]	Loss 0.80292	Acc 0.72427	Precision 0.57392	Recall 0.22759	F1 0.32593
Epoch: [4][60/278]	Loss 0.79992	Acc 0.72487	Precision 0.57537	Recall 0.22865	F1 0.32726
Epoch: [4][70/278]	Loss 0.79733	Acc 0.72508	Precision 0.57695	Recall 0.23092	F1 0.32982
Epoch: [4][80/278]	Loss

Test: [0/21]	Loss 0.67853	Acc 0.66683	Precision 0.73462	Recall 0.24116	F1 0.36312
Test: [10/21]	Loss 0.63269	Acc 0.69139	Precision 0.73462	Recall 0.24116	F1 0.36312
Test: [20/21]	Loss 0.70729	Acc 0.66208	Precision 0.74955	Recall 0.22348	F1 0.34431
 * Acc 0.66208 F1 0.34431
Epoch: [7][0/278]	Loss 0.68430	Acc 0.74080	Precision 0.60578	Recall 0.32951	F1 0.42684
Epoch: [7][10/278]	Loss 0.68395	Acc 0.74038	Precision 0.60406	Recall 0.32974	F1 0.42661
Epoch: [7][20/278]	Loss 0.68309	Acc 0.74057	Precision 0.60459	Recall 0.33087	F1 0.42768
Epoch: [7][30/278]	Loss 0.68214	Acc 0.74086	Precision 0.60587	Recall 0.33243	F1 0.42931
Epoch: [7][40/278]	Loss 0.68120	Acc 0.74097	Precision 0.60587	Recall 0.33312	F1 0.42988
Epoch: [7][50/278]	Loss 0.68027	Acc 0.74112	Precision 0.60645	Recall 0.33396	F1 0.43072
Epoch: [7][60/278]	Loss 0.67920	Acc 0.74134	Precision 0.60699	Recall 0.33440	F1 0.43123
Epoch: [7][70/278]	Loss 0.67840	Acc 0.74154	Precision 0.60726	Recall 0.33499	F1 0.43179
Epoch: [7][80/278]	Loss

Test: [0/21]	Loss 0.67845	Acc 0.67758	Precision 0.71888	Recall 0.30135	F1 0.42467
Test: [10/21]	Loss 0.64559	Acc 0.69386	Precision 0.71888	Recall 0.30135	F1 0.42467
Test: [20/21]	Loss 0.70768	Acc 0.67429	Precision 0.72921	Recall 0.28561	F1 0.41045
 * Acc 0.67429 F1 0.41045
Epoch: [10][0/278]	Loss 0.62760	Acc 0.75073	Precision 0.62180	Recall 0.38081	F1 0.47234
Epoch: [10][10/278]	Loss 0.62724	Acc 0.75069	Precision 0.62148	Recall 0.38083	F1 0.47226
Epoch: [10][20/278]	Loss 0.62685	Acc 0.75075	Precision 0.62122	Recall 0.38173	F1 0.47288
Epoch: [10][30/278]	Loss 0.62630	Acc 0.75100	Precision 0.62187	Recall 0.38288	F1 0.47395
Epoch: [10][40/278]	Loss 0.62595	Acc 0.75093	Precision 0.62163	Recall 0.38314	F1 0.47408
Epoch: [10][50/278]	Loss 0.62521	Acc 0.75114	Precision 0.62201	Recall 0.38338	F1 0.47437
Epoch: [10][60/278]	Loss 0.62469	Acc 0.75129	Precision 0.62230	Recall 0.38388	F1 0.47484
Epoch: [10][70/278]	Loss 0.62430	Acc 0.75137	Precision 0.62237	Recall 0.38443	F1 0.47528
Epoch: [10][80/

Epoch: [12][270/278]	Loss 0.59512	Acc 0.75776	Precision 0.63135	Recall 0.41607	F1 0.50159
Test: [0/21]	Loss 0.69420	Acc 0.68647	Precision 0.76115	Recall 0.30177	F1 0.43219
Test: [10/21]	Loss 0.66905	Acc 0.69851	Precision 0.76115	Recall 0.30177	F1 0.43219
Test: [20/21]	Loss 0.71193	Acc 0.67947	Precision 0.76209	Recall 0.28001	F1 0.40955
 * Acc 0.67947 F1 0.40955
Epoch: [13][0/278]	Loss 0.59489	Acc 0.75781	Precision 0.63144	Recall 0.41618	F1 0.50169
Epoch: [13][10/278]	Loss 0.59494	Acc 0.75776	Precision 0.63135	Recall 0.41661	F1 0.50198
Epoch: [13][20/278]	Loss 0.59458	Acc 0.75788	Precision 0.63174	Recall 0.41677	F1 0.50222
Epoch: [13][30/278]	Loss 0.59422	Acc 0.75803	Precision 0.63211	Recall 0.41721	F1 0.50266
Epoch: [13][40/278]	Loss 0.59395	Acc 0.75804	Precision 0.63232	Recall 0.41755	F1 0.50296
Epoch: [13][50/278]	Loss 0.59359	Acc 0.75811	Precision 0.63242	Recall 0.41801	F1 0.50333
Epoch: [13][60/278]	Loss 0.59311	Acc 0.75828	Precision 0.63266	Recall 0.41859	F1 0.50383
Epoch: [13][70

Epoch: [15][260/278]	Loss 0.57603	Acc 0.76087	Precision 0.63690	Recall 0.42812	F1 0.51205
Epoch: [15][270/278]	Loss 0.57567	Acc 0.76104	Precision 0.63710	Recall 0.42824	F1 0.51220
Test: [0/21]	Loss 0.69344	Acc 0.68892	Precision 0.78140	Recall 0.29697	F1 0.43038
Test: [10/21]	Loss 0.67638	Acc 0.69846	Precision 0.78088	Recall 0.29697	F1 0.43030
Test: [20/21]	Loss 0.68911	Acc 0.69135	Precision 0.79156	Recall 0.30208	F1 0.43729
 * Acc 0.69135 F1 0.43729
Epoch: [16][0/278]	Loss 0.57552	Acc 0.76116	Precision 0.63739	Recall 0.42860	F1 0.51255
Epoch: [16][10/278]	Loss 0.57519	Acc 0.76135	Precision 0.63776	Recall 0.42887	F1 0.51286
Epoch: [16][20/278]	Loss 0.57497	Acc 0.76134	Precision 0.63763	Recall 0.42898	F1 0.51289
Epoch: [16][30/278]	Loss 0.57483	Acc 0.76134	Precision 0.63749	Recall 0.42908	F1 0.51292
Epoch: [16][40/278]	Loss 0.57465	Acc 0.76135	Precision 0.63746	Recall 0.42937	F1 0.51312
Epoch: [16][50/278]	Loss 0.57447	Acc 0.76137	Precision 0.63754	Recall 0.42929	F1 0.51309
Epoch: [16][6

In [6]:
import torch.utils.model_zoo as model_zoo
model_dict = model_zoo.load_url('https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth', '/home/krf/model/BALL/')

In [10]:
print(model_dict.keys())

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])


In [16]:
model = tcnn(2)
pre_dict = model.state_dict()
print(pre_dict.keys())

AttributeError: 'odict_keys' object has no attribute 'values'

In [ ]:
k1 = ['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias']
k2 = ['conv1.0.weight', 'conv1.0.bias', 'conv2.0.weight', 'conv2.0.bias', 'conv3.0.weight', 'conv3.0.bias', 'conv4.0.weight', 'conv4.0.bias', 'conv5.0.weight', 'conv5.0.bias', 'fc1.weight', 'fc1.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.3.weight', 'classifier.3.bias']
for i in range(len(k1)):
    pretrained_dict[k2[i]] = model_dict[k1[i]]

# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)